## softmax

In [2]:
import torch
import torch.nn as nn

In [2]:
y = torch.tensor([0,2,1,0])
score = torch.tensor([[1,1,1],
                      [0,0,0],
                      [2,2,2],
                      [2,2,2]],dtype=torch.float32)

#### right result
当每个类别得分都一样的时候，很容易得到softmax结果为

In [5]:
import numpy as np
-np.log(1./3)

1.0986122886681098

#### softmax formula

In [12]:
score_exp = torch.exp(score)
score_correct = score_exp[range(4),y]
loss = -torch.log(score_correct/torch.sum(score_exp,dim=1))
loss = torch.sum(loss)/(4)
print(loss)

tensor([[2.7183, 2.7183, 2.7183],
        [1.0000, 1.0000, 1.0000],
        [7.3891, 7.3891, 7.3891],
        [7.3891, 7.3891, 7.3891]])
tensor([2.7183, 1.0000, 7.3891, 7.3891])
tensor(1.0986)


#### pytorch API

In [23]:
y_onehot = torch.zeros(4,3).type(torch.LongTensor)
y_onehot[range(4),y] = 1
print(y_onehot)
loss_fun = nn.CrossEntropyLoss()
loss = loss_fun(score,y)
#loss = loss_fun(score,y_onehot)   # error
print(loss)

tensor([[1, 0, 0],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0]])
tensor(1.0986)


## Autograd

In [48]:
torch.manual_seed(1)    # reproducible
X = torch.randn(4,3)         # samples=4, features=3
y = torch.tensor([0,2,1,1])  # classes=3
W = torch.randn(3,3,requires_grad=True)        
b = torch.randn(1,requires_grad=True)

score = torch.mm(X,W) + b
#------- (2)formular -----------
score_exp = torch.exp(score)
score_correct = score_exp[range(4),y]
loss = -torch.log(score_correct/torch.sum(score_exp,dim=1))
loss = torch.sum(loss)/(4)

#------- (1)with API -----------
#loss_fun = nn.CrossEntropyLoss()
#loss = loss_fun(score,y)

print(loss)

tensor(0.9580, grad_fn=<DivBackward0>)


#### softmax formula

In [51]:
with torch.no_grad():
    score_correct = score[range(4),y]
    score_exp     = torch.exp(score)
    score_total   = torch.sum(score_exp,dim=1)
    
    ratio_label   = torch.zeros(4,3)
    ratio_label[range(4),y] = 1
    dscore        = (-ratio_label + score_exp/score_total.view(-1,1))/4
    dW = torch.mm(X.t(),dscore)
    print(dW)

tensor([[-0.3271,  0.3744, -0.0472],
        [-0.0770,  0.0185,  0.0586],
        [-0.1946,  0.1629,  0.0316]])


#### pytorch API

In [50]:
loss.backward()
print(W.grad)

tensor([[-0.3271,  0.3744, -0.0472],
        [-0.0770,  0.0185,  0.0586],
        [-0.1946,  0.1629,  0.0316]])


## sigmoid

In [5]:
# x is the image
x = torch.randn(1,3,5,5)
x.size()

torch.Size([1, 3, 5, 5])

In [7]:
conv = torch.nn.Conv2d(3,2,kernel_size=1)
score = conv(x)
score.size()

torch.Size([1, 2, 5, 5])

#### sigmoid with pytorch API

In [16]:
output_api = torch.sigmoid(score)
output_api

tensor([[[[0.4086, 0.3859, 0.4177, 0.4022, 0.3978],
          [0.4485, 0.3928, 0.4195, 0.2599, 0.3833],
          [0.3592, 0.4228, 0.4180, 0.3212, 0.3197],
          [0.4238, 0.3536, 0.2798, 0.3790, 0.4303],
          [0.2812, 0.3690, 0.3532, 0.4140, 0.4503]],

         [[0.5989, 0.4652, 0.5822, 0.4564, 0.5941],
          [0.5758, 0.4407, 0.5424, 0.4572, 0.5406],
          [0.5604, 0.6901, 0.6262, 0.4823, 0.4444],
          [0.4804, 0.4853, 0.4401, 0.4702, 0.6555],
          [0.5444, 0.5608, 0.3800, 0.5361, 0.6534]]]],
       grad_fn=<SigmoidBackward>)

#### sigmoid with formula

In [17]:
output_formular = 1/(1 + torch.exp(-score))
output_formular

tensor([[[[0.4086, 0.3859, 0.4177, 0.4022, 0.3978],
          [0.4485, 0.3928, 0.4195, 0.2599, 0.3833],
          [0.3592, 0.4228, 0.4180, 0.3212, 0.3197],
          [0.4238, 0.3536, 0.2798, 0.3790, 0.4303],
          [0.2812, 0.3690, 0.3532, 0.4140, 0.4503]],

         [[0.5989, 0.4652, 0.5822, 0.4564, 0.5941],
          [0.5758, 0.4407, 0.5424, 0.4572, 0.5406],
          [0.5604, 0.6901, 0.6262, 0.4823, 0.4444],
          [0.4804, 0.4853, 0.4401, 0.4702, 0.6555],
          [0.5444, 0.5608, 0.3800, 0.5361, 0.6534]]]], grad_fn=<MulBackward0>)

### BCELoss
Binary Cross Entropy Loss

In [21]:
# x is the input image
x = torch.randn(1,3,5,5)     
# y is the target
y = torch.zeros(1,2,5,5)     # two class
y[:,0,:2,:] = 1
y[:,1,2:,:] = 1
conv = torch.nn.Conv2d(3,2,kernel_size=1)
score = conv(x)
output = torch.sigmoid(score)

#### BCELoss with pytorch API

In [23]:
loss_fun = torch.nn.BCELoss()
loss_api = loss_fun(output,y)
loss_api

tensor(0.7098, grad_fn=<BinaryCrossEntropyBackward>)

#### BCELoss with formula

In [62]:
score = torch.tensor([5.,8.])
output_y = torch.sigmoid(score)
target_y = torch.tensor([0.,1])
loss_fun = torch.nn.BCELoss()
loss = loss_fun(output_y,target_y)
loss

tensor(2.5035)

第一种计算方式

In [65]:
loss0 = (target_y[0]*torch.log(output_y[0]) + (1-target_y[0])*torch.log(1-output_y[0]))
loss1 = (target_y[1]*torch.log(output_y[1]) + (1-target_y[1])*torch.log(1-output_y[1]))
loss = -(loss0+loss1)/2
loss

tensor(2.5035)

第二种计算方式

In [64]:
loss = target_y*torch.log(output_y) + (1-target_y)*torch.log(1-output_y)
loss = -torch.sum(loss)/2
loss

tensor(2.5035)